In [ ]:
#grouping two datasets by videoid first
import pandas as pd
import numpy as np

# Load the latest processed comments (from your spam detection pipeline)
comments_df = pd.read_csv('../complete_comments_with_features.csv')
print(f"Loaded comments: {len(comments_df):,} rows")

# Load original videos dataset
videos_df = pd.read_csv('../dataset/videos.csv')
print(f"Loaded videos: {len(videos_df):,} rows")

# Check unique videoIds in each dataset
unique_videos_in_comments = comments_df['videoId'].nunique()
unique_videos_in_videos = videos_df['videoId'].nunique()

print(f"\nUnique videoIds in comments: {unique_videos_in_comments:,}")
print(f"Unique videoIds in videos: {unique_videos_in_videos:,}")

# Find videos without comments and comments without videos
videos_with_comments = set(comments_df['videoId'].unique())
videos_in_dataset = set(videos_df['videoId'].unique())

videos_without_comments = videos_in_dataset - videos_with_comments
comments_without_videos = videos_with_comments - videos_in_dataset

print(f"\nMERGE STATISTICS:")
print(f"Videos without any comments: {len(videos_without_comments):,}")
print(f"Comments referencing non-existent videos: {len(comments_without_videos):,}")

# Show some examples if they exist
if videos_without_comments:
    print(f"\nExample videos without comments: {list(videos_without_comments)[:5]}")
if comments_without_videos:
    print(f"Example videoIds in comments but not in videos: {list(comments_without_videos)[:5]}")

# Perform the merge - keeping all comments that have matching videos
merged_df = comments_df.merge(videos_df, on='videoId', how='inner', suffixes=('_comment', '_video'))

print(f"\nMERGE RESULTS:")
print(f"Comments before merge: {len(comments_df):,}")
print(f"Comments after merge: {len(merged_df):,}")
print(f"Comments lost in merge: {len(comments_df) - len(merged_df):,}")
print(f"Videos included in merge: {merged_df['videoId'].nunique():,}")

# Show what happens to comment IDs - they are preserved!
print(f"\nComment IDs preserved: All {len(merged_df):,} comment IDs remain intact")
print(f"Each row still represents one comment, now enriched with video metadata")

print(f"\nMERGED DATASET COLUMNS:")
print("Comment columns:", [col for col in merged_df.columns if col.endswith('_comment') or col in ['commentId', 'textOriginal', 'likeCount_comment']])
print("Video columns:", [col for col in merged_df.columns if col.endswith('_video') or col in ['title', 'description', 'tags']])
print("Shared columns:", [col for col in merged_df.columns if not col.endswith('_comment') and not col.endswith('_video') and col != 'videoId'])

# Display merged dataframe in table format
print("\nSAMPLE OF MERGED DATA:")
print("="*100)

# Select key columns for easier viewing
key_columns = ['commentId', 'videoId', 'textOriginal', 'title', 'description', 'likeCount_comment', 'viewCount']
display_df = merged_df[key_columns].head()

# Set pandas display options for better table formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print(display_df.to_string(index=False))
print("="*100)